In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))


def der_sigmoid(x):
    return np.exp(-x)/((1 + np.exp(-x)) ** 2)

In [7]:
def der_log_loss(y_hat, y_true):
    """
    0 < y_hat < 1
    """
    return y_true/y_hat - (1-y_true)/(1-y_hat)

In [12]:
def calc_gradient(y_hat, y_true):
    return der_log_loss(sigmoid(y_hat), y_true) * der_sigmoid(y_hat)

Теперь мы можем написать код градиентного бустинга для классификации

In [13]:
from sklearn.base import BaseEstimator # чтобы поддержать интерфейс sklearn
from sklearn.tree import DecisionTreeRegressor # для обучения на каждой итерации

In [102]:
class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters=100, tau=1e-1):
        """
        tree_params_dict - словарь параметров, которые надо использовать при обучении дерева на итерации
        iters - количество итераций
        tau - коэффициент перед предсказаниями деревьев на каждой итерации
        """
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        
    def fit(self, X_data, y_data):
        self.estimators = []
        curr_pred = 0
        for iter_num in range(self.iters):
            # Нужно найти градиент функции потерь по предсказниям в точке curr_pred
            grad = calc_gradient(curr_pred, y_data) # TODO
            # Мы максимизируем, поэтому надо обучить DecisionTreeRegressor с параметрами 
            # tree_params_dict по X_data предсказывать grad
            algo = DecisionTreeRegressor(max_depth = 4).fit(X_data,grad) # TODO
            self.estimators.append(algo)
            # все предсказания домножаются на tau и обновляется переменная curr_pred
            curr_pred += self.tau * algo.predict(X_data)

    def predict(self, X_data):
        # изначально все предскзания нули
        
        res = np.zeros(X_data.shape[0])
       
        for estimator in self.estimators:     
            # нужно сложить все предсказания деревьев с весом self.tau
            pred = estimator.predict(X_data)
            #print(pred)     
            for i,pr in enumerate(pred):
                #if pr % self.tau == 0:
                res[i] += pr *self.tau       
        #print(res)   
        return (res > 0).astype(int)

In [103]:
# для оценки качества
from sklearn.model_selection import cross_val_score

# для генерации датасетов
from sklearn.datasets import make_classification

# для сравнения
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [104]:
X_data, y_data = make_classification(n_samples=1000, n_features=10, random_state=42)

In [105]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth':4
    },
    iters=100,
    tau = 0.1
)

In [106]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='accuracy'))

0.921

In [68]:
np.mean(cross_val_score(DecisionTreeClassifier(), X_data, y_data, cv=5, scoring='accuracy'))

0.8610187004675117

In [69]:
np.mean(cross_val_score(XGBClassifier(), X_data, y_data, cv=5, scoring='accuracy'))

0.8969841246031149

In [70]:
np.mean(cross_val_score(LogisticRegression(), X_data, y_data, cv=5, scoring='accuracy'))

0.8560087002175054